This notebook modifies the wrangling step to work with open data. The paper depends on an `.Rdata` file (that is not publicly available) that contains  a set of 10 separate dataframes per product (i.e. category) -- see Table 1 in the paper for a summary stats by product. To work with the [open New Zealand data](https://code.officialstatistics.org/scanner-task-team-gwg/FEWS_package/-/blob/master/data/SampleDataSet.csv?ref_type=heads) that is similar - I modified the wrangling step.

In [ ]:
# | echo: false
# | warning: false
# | output: false

library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.5.2
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## 1. Overview of requirements

### Research expected input

The `.Rdata` file contains 10 dataframes - each has the following variables:
- `period`, Date format e.g. "2021-06-01"
- `price`, num format
- `volume`, num format
- `value`, num format
- `id`, Factor format

### Data input that is available 

The `csv` dataset is a little different:
* `month_num`
* `char1` - `char11` (11 char variables of various kinds) 
* `prodid_num`
* `quantity`
* `value`

We don't have a `price` variable but it may be applicable to make it from `value/quantity` (although this is an assumption). See the fuller [dataset exploration](https://sergegoussev.github.io/price_stats_experiments/notebooks/nz_electronics_2019.html) for more info.  

### Steps to take

The wrangling script [qualdecomp_1wrangle.R](https://github.com/sergegoussev/quality_decomposition/blob/main/R/qualdecomp_1wrangle.R) takes each and computes the following:
1. Calculate expenditure shares per product as a proportion of the total expenditure on all products.
2. Derives the log of price
3. Makes the period a factor for the TPD


## Replicating the process with another dataset

The following steps are then followed in order to replicate the wrangling process with the open NZ data.

### Step 1a. Load the raw data

In [ ]:
#| echo: true

alldata <- read.csv("../data/raw/NZ_dataset_historic_raw.csv", header = TRUE)
head(alldata)

,month_num,char11,char1,char2,char3,char4,char5,char6,char7,char8,char9,char10,prodid_num,quantity,value
,<chr>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>
1,2017-01-01,brand_a,10.6,16006,val_w,val_a,val_a,PRG566,CCC,150D,B230,ted,3,280,196420
2,2017-02-01,brand_a,10.6,16006,val_w,val_a,val_a,PRG566,CCC,150D,B230,ted,3,126,85312
3,2017-03-01,brand_a,10.6,16006,val_w,val_a,val_a,PRG566,CCC,150D,B230,ted,3,148,95920
4,2017-04-01,brand_a,10.6,16006,val_w,val_a,val_a,PRG566,CCC,150D,B230,ted,3,56,38552
5,2017-05-01,brand_a,10.6,16006,val_w,val_a,val_a,PRG566,CCC,150D,B230,ted,3,69,47397
6,2017-06-01,brand_a,10.6,16006,val_w,val_a,val_a,PRG566,CCC,150D,B230,ted,3,43,28303


### Step 1b. Format data to be by product

In [35]:
#| echo: true

product <- alldata %>%
    subset(select = c(month_num, prodid_num, quantity, value)) %>%
    mutate(expenditure = value) %>%
    mutate(price = value/quantity) %>%
    subset(select = -c(value)) %>%
    rename(i = prodid_num, period = month_num)
head(product)

,period,i,quantity,expenditure,price
,<chr>,<int>,<int>,<int>,<dbl>
1,2017-01-01,3,280,196420,701.5000
2,2017-02-01,3,126,85312,677.0794
3,2017-03-01,3,148,95920,648.1081
4,2017-04-01,3,56,38552,688.4286
5,2017-05-01,3,69,47397,686.9130
6,2017-06-01,3,43,28303,658.2093


### Step 2. Get the expenditure shares, log prices, and factor periods

In [36]:
#| echo: true

tot_exps <- product %>%
  group_by(period) %>%
  summarise(tot_exp = sum(expenditure))

product_s_l <- merge(product, tot_exps, by = "period", all = TRUE) %>%
  mutate(exp_share = expenditure/tot_exp,
         logprice = log(price)) %>%
  subset(select = -c(tot_exp)) 

product_s_l$period <- as.factor(product_s_l$period)

head(product_s_l)

,period,i,quantity,expenditure,price,exp_share,logprice
,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,2017-01-01,3,280,196420,701.5000,1.562999e-02,6.553221
2,2017-01-01,366,1,1009,1009.0000,8.029052e-05,6.916715
3,2017-01-01,1004,33,52688,1596.6061,4.192614e-03,7.375635
4,2017-01-01,1004,1,1221,1221.0000,9.716029e-05,7.107425
5,2017-01-01,741,230,305802,1329.5739,2.433400e-02,7.192614
6,2017-01-01,943,99,85891,867.5859,6.834721e-03,6.765714


### Save the processed data

To help replicate other aspects of the research - save the processed dataset to `/data/processed/`. 

In [58]:
write.csv(product_s_l, "../data/processed/product_s_1_raw.csv", row.names = FALSE)